In [ ]:
import xarray as xr
import data_types
import importlib
import matplotlib.pyplot as plt
import numpy as np

years=range(2020,2021)

# data_dir = '/scratch/cimes/feiyul/Ocean_Data'
data_dir = '/collab1/data_untrusted/Feiyu.Lu/Ocean_Data'
T_varname = 'thetao_prior_z'
S_varname = 'so_prior_z'
SST_obs_varname = 'sst'
SSS_obs_varname = 'sss'

ocean_static_ds = xr.open_dataset(f'{data_dir}/model_data/M9/ocean_z.static.nc')
ocean_static_ds['xh']=ocean_static_ds['xh'].where(ocean_static_ds['xh']>0,ocean_static_ds['xh']+360)
ocean_static_ds['xq']=ocean_static_ds['xq'].where(ocean_static_ds['xq']>0,ocean_static_ds['xq']+360)
ocean_static_ds_rolled = ocean_static_ds.roll(xh=60,roll_coords=True).roll(xq=60,roll_coords=True)
geolat= ocean_static_ds_rolled.geolat
geolon= ocean_static_ds_rolled.geolon

ocean_levels_ds = xr.open_dataset(f'{data_dir}/model_data/M9/vgrid_75_2m.nc')
ocean_levels = ocean_levels_ds.dz

T_ds = xr.open_mfdataset(
    [f'{data_dir}/model_data/M9/ocean_daily.{year}0101-{year}1231.{T_varname}.nc' for year in years])
T_ds['xh']=T_ds['xh'].where(T_ds['xh']>0,T_ds['xh']+360)
T_ds_rolled = T_ds.roll(xh=60,roll_coords=True)
T_var = T_ds_rolled[T_varname]

SST_obs_ds = xr.open_mfdataset(
    [f'{data_dir}/obs_data/sst/sst.day.{year}.1x1.nc' for year in years])
SST_obs_var = SST_obs_ds[SST_obs_varname]

SSS_obs_ds = xr.open_mfdataset(
    [f'{data_dir}/obs_data/sss/SSS.day.{year}.1x1.nc' for year in years])
SSS_obs_var = SSS_obs_ds[SSS_obs_varname]

argo_obs_ds = xr.open_mfdataset(
    [f'{data_dir}/obs_data/argo/argo_{year}.nc' for year in years])

In [ ]:
argo_obs_ds

In [ ]:
T_var

In [ ]:
SST_obs_var

In [ ]:
importlib.reload(data_types)
argo_obs = data_types.argo(argo_obs_ds,nread=10000,nstart=80000)
len(argo_obs)

In [ ]:
print(argo_obs.profiles[-1].time)
print(argo_obs.time[-1])
print(argo_obs.lat[-1])
print(argo_obs.lon[-1])
plt.plot(argo_obs.profiles[-1].T, argo_obs.profiles[-1].depth,label='Argo T')

In [ ]:
importlib.reload(data_types)

indexes = argo_obs.get_profiles_by_period(np.datetime64('2020-01-03T00:00:00'),np.datetime64('2020-01-08T00:00:00'))
print(indexes)
print(argo_obs.lat[indexes])

In [ ]:
from cartopy import crs as ccrs
import matplotlib.pyplot as plt
import seaborn

fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(12,12),
                         subplot_kw={'projection': ccrs.Robinson(210)})

day = 5
period = [np.datetime64('2020-01-03T00:00:00'),
          np.datetime64('2020-01-08T00:00:00')]
indexes = argo_obs.get_profiles_by_period(period[0], period[1])
level = 0
cmap = seaborn.color_palette("Spectral_r", as_cmap=True)

axes[0].pcolormesh(geolon, geolat, T_var.isel(z=level,time=day), vmin=-2, vmax=32, cmap=cmap,
                   transform = ccrs.PlateCarree())

axes[1].pcolormesh(SST_obs_var.isel(time=day), vmin=-2, vmax=32, cmap=cmap,
                   transform = ccrs.PlateCarree())
axes[1].scatter(argo_obs.lon[indexes], argo_obs.lat[indexes], 
                s=5, transform = ccrs.PlateCarree())
# axes[1].
# fig.tight_layout()